In [ ]:
"""

Get data for features from each of the county Wikipedia pages
Loop over list of URLs of Wikipedia pages for each US county 

features to collect:
highways
cities
towns

output to pickle

Run time: about 45 min

"""

In [1]:
from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd

import pickle
import time

import regex as re

In [2]:
# list of county page wikipedia links 

with open('county_wiki_links.pickle', 'rb') as read_file:
    county_links = pickle.load(read_file)

In [3]:
county_links.head(5)

,state,FIPS_code,county_name,wiki_link
0,Alabama,001,"Autauga County, Alabama","/wiki/Autauga_County,_Alabama"
1,Alabama,003,"Baldwin County, Alabama","/wiki/Baldwin_County,_Alabama"
2,Alabama,005,"Barbour County, Alabama","/wiki/Barbour_County,_Alabama"
3,Alabama,007,"Bibb County, Alabama","/wiki/Bibb_County,_Alabama"
4,Alabama,009,"Blount County, Alabama","/wiki/Blount_County,_Alabama"


In [5]:
len(county_links)

3094

In [4]:
url = 'https://en.wikipedia.org'

In [6]:
records=[]

for i, county in enumerate(county_links['county_name']):


    state = county_links['state'][i]
    county_name = county_links['county_name'][i]
    FIPS_code = county_links['FIPS_code'][i]

    county_page_url = url + county_links['wiki_link'][i]

    response = requests.get(county_page_url)
    page = response.text
    soup = BeautifulSoup(page)

    try:

        try:
            highway_list = soup.find(id=re.compile('highway')).findNext('ul').find_all('a')   
            highways = [x.text for x in highway_list[1::2]]
            hwy_count = len(highways)

        except:
            highways = []
            hwy_count = 0

        try:
            city_list = soup.find(id=re.compile('Cit')).findNext('ul').find_all('li')
            cities = [x.find('a').text for x in city_list]
            city_count = len(cities)

        except:
            cities = []
            cities_count = 0

        try:
            town_list = soup.find(id=re.compile('Town')).findNext('ul').find_all('li')
            towns = [x.find('a').text for x in town_list]
            town_count= len(towns)

        except:
            towns = []
            town_count = 0

        records.append((state, county_name, FIPS_code, county_page_url, highways, hwy_count,
                    cities, city_count, towns, town_count))

    except:
        print('ERROR with URL: ', state_page_url)
        continue


    time.sleep(1)
        
  


In [7]:
len(records)

3094

In [13]:
# to df
county_features = pd.DataFrame(records)
county_features.columns = ['state', 'county_name', 'FIPS_code', 'county_page_url', 'highways', 'hwy_count',
                    'cities', 'city_count', 'towns', 'town_count']

In [14]:
len(county_features)

3094

In [16]:
# output pickle
with open('county_features.pickle', 'wb') as to_write:
    pickle.dump(county_features, to_write)

In [15]:
county_features.to_csv('review_features_scrape.csv', index=False)